**How this will work** <br>


1.   A user will provide a hypothesis and a document and the model will then infer if there is any clause in the document that support, contradict or not mentioned(neutral) the hypothesis
2.   Model will be batch trained. Each batch will represent one document and one hypothesis. 
> Note that in a document there can be 100's of clauses but just one or two clause may support or contradict the hypothesis. This implies over 90% of clauses will be neutral to the hypothesis. This makes data highly imbalanced <br>
> I have a parameter max_neutral that controles how many neutral cases we want to add






In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
dir= "/content/drive/MyDrive/GT/Deep_Learning/Project/contract-nli/contract-nli/"
os.chdir(dir)

In [ ]:
!apt-get -qq install -y libfluidsynth1
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
!apt-get -qq install -y graphviz && pip install pydot
#!pip install cartopy
!apt-get install libomp-dev

Selecting previously unselected package libfluidsynth1:amd64.
(Reading database ... 123991 files and directories currently installed.)
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.6) ...
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 123996 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.2.2-3.1ubuntu0.7_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.2.2-3.1ubuntu0.7) ...
Setting up libarchive-dev:amd64 (3.2.2-3.1ubuntu0.7) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 154 kB 6.9 MB/s 
  Created wheel for libarchive: filename=libarchive-0.4.7-py3-none-any.whl size=31646 sha256=60b31f0b99a28ccdc71da9b8ba3c9ced33b

In [ ]:
!pip install torch
!pip install sentence-transformers
!pip install pdf_struct
!pip install faiss
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 5.5 MB 12.2 MB/s 
     |████████████████████████████████| 1.3 MB 38.9 MB/s 
     |████████████████████████████████| 163 kB 24.6 MB/s 
     |████████████████████████████████| 7.6 MB 41.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=54d11f2012130262ec2d869c583e41be517c5685fc19f08d5824fb4533f46a5f
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.2 MB/s 


In [ ]:
import json
import pickle
import pandas as pd
import re 
import transformers
import torch
import numpy as np
#from torchtext.data import get_tokenizer
#from transformers import BertTokenizer, BertModel, BertConfig
from torch.utils.data import Dataset, DataLoader

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from sklearn.metrics import pairwise

d=768 # Do not change



In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
nltk.download('omw-1.4')
import re
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

mlngt = 100 #max_length,
mn =10 #max_neutral

def clean_head(ab):
    ab = ab.replace(","," ,").replace(";"," ;").replace(":"," :")
    ab = ab.lower().strip()    
    #marked_text = "[CLS] " + ab + " [SEP]"
    # Tokenize our sentence with the BERT tokenizer.
    #ab = tokenizer.tokenize(marked_text) 
    return ab

def preprocess(sentence):
  sentence=str(sentence)
  sentence = sentence.lower().replace(","," ,").replace(";"," ;").replace(":"," :").replace("__"," ").replace("  "," ")
  sentence=sentence.replace('{html}',"") 
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', sentence)
  rem_url=re.sub(r'http\S+', '',cleantext)
  rem_num = re.sub('[0-9]+', '', rem_url)
  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(rem_num)  
  #filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
  #stem_words=[stemmer.stem(w) for w in filtered_words]
  #lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
  return " ".join(tokens)



def getPositionEncoding(max_length, d, n=10000):
    P = np.zeros((max_length, d))
    #print(max_length,P.shape)
    for k in range(max_length):      
      for i in np.arange(int(d/2)):
        denominator = np.power(n, 2*i/d)
        P[k, 2*i] = np.sin(k/denominator)
        P[k, 2*i+1] = np.cos(k/denominator)
    return P
Pg =  getPositionEncoding(mlngt, d)  
Pg  = torch.FloatTensor(Pg)

def emd(sent, max_length=mlngt):
  tok = get_tokenizer("basic_english")
  tkn = tok(sent)  
  ln = len(tkn)
  x = global_vectors.get_vecs_by_tokens(tkn)
  #print(x.shape, Pg.shape)
  if ln<max_length:  
    x = torch.nn.ConstantPad2d((0, 0, 0, max_length-ln), 0)(x)
  else:
    x= x[:max_length,:]
  x=x+Pg 
  return x

def emdb(sent):
  # Tokenize sentences
  encoded_input = tokenizer(sent, padding=True, truncation=True, return_tensors='pt')

  # Compute token embeddings
  with torch.no_grad():
      model_output = model(**encoded_input)

  # Perform pooling. In this case, max pooling.
  x = mean_pooling(model_output, encoded_input['attention_mask'])
  seg = torch.tensor([0,1])
  seg = seg.tile((768,1)).t()    
  x=x+seg  
  return x


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
#################
# Opening JSON file
f = open('train.json')  
# returns JSON object as 
# a dictionary
data = json.load(f)
flnmidx={}
for dcl in range(len(data['documents'])):
  flnmidx[data['documents'][dcl]['file_name']]= dcl


lbl={}
kes =[]
for ke in data['labels']:
  lbl[ke]= data['labels'][ke]['hypothesis']
  kes.append(ke)

kes=['nda-11', 'nda-16', 'nda-15', 'nda-10', 'nda-2', 'nda-1', 'nda-19', 'nda-12', 'nda-20', 'nda-3', 'nda-18', 'nda-7', 'nda-17', 'nda-8', 'nda-13', 'nda-5', 'nda-4']
rg = len(data['documents'])
reference_list=[]
idx=0
for idx in range(rg):
  #Document level
  docid = data['documents'][idx]['id']
  for ndas_key in kes:
    reference_list.append([docid, ndas_key,idx])
  idx=idx+1
ref_train = pd.DataFrame(reference_list ,columns = ['docid', 'nda_key', 'idx'])
f.close()
##############
# Opening JSON file
f = open('test.json')  
# returns JSON object as 
# a dictionary
dataT = json.load(f)

kes=['nda-11', 'nda-16', 'nda-15', 'nda-10', 'nda-2', 'nda-1', 'nda-19', 'nda-12', 'nda-20', 'nda-3', 'nda-18', 'nda-7', 'nda-17', 'nda-8', 'nda-13', 'nda-5', 'nda-4']
rg = len(dataT['documents'])
reference_list=[]
idxt=0
for idxt in range(rg):
  #Document level
  docid = dataT['documents'][idxt]['id']

  for ndas_key in kes:
    reference_list.append([docid, ndas_key, idxt])
  idxt=idxt+1
ref_test = pd.DataFrame(reference_list ,columns = ['docid', 'nda_key', 'idx'])

In [ ]:
# Do Not delete
def get_data(data, idx,ke, max_neutral=mn):
  dataM=[]
  #Document level
  docid = data['documents'][idx]['id']
  
  string = data['documents'][idx]['text']
  file_name = data['documents'][idx]['file_name']
  spans = data['documents'][idx]['spans']
  spanall=[]
  for span in spans:           
      spanval = string[span[0]: span[1]]
      spanall.append(spanval)
  
  ndas = data['documents'][idx]['annotation_sets'][0]['annotations'] 
  # Keys level
    
  hypothesis = data['labels'][ke]['hypothesis']  

  # Key level   

  choice =ndas[ke] ['choice']
  spansC = ndas[ke] ['spans']    
  for si in range(len(spanall)):
    span_nbr =si
    if si in spansC:
      val=choice
    else:
      val="Neutral"
    premise = spanall[si]
    itm = [docid, file_name , ke, hypothesis, span_nbr, premise,val ]
    dataM.append(itm)
  df = pd.DataFrame(dataM ,columns = ['docid', 'file_name', 'hypotheis_key', 'hypotheis', 'span_nbr', 'premise', 'choice'   ])
  #"[CLS] " and " [SEP] "" [SEP]" 
  df['premise'] = df['premise'].apply(preprocess)
  df['hypotheis']= df['hypotheis'].apply(preprocess) 
  df['label'] = df['choice'].map(lambda s: 0 if s=='Entailment' else (1 if s== 'Contradiction' else 2 ))
  df['entl'] =  df['choice'].map(lambda s : 1 if s== 'Entailment' else 0)
  df['cont'] =  df['choice'].map(lambda s : 1 if s== 'Contradiction' else 0)
  df['neut'] =  df['choice'].map(lambda s : 1 if s== 'Neutral' else 0)
  
  df1=df[df.choice !='Neutral']
  df2=df[df.choice =='Neutral']
  n = min(max_neutral, len(df2)-1)
  df2= df2.sample(n = n)
  df = pd.concat([df1, df2], sort=False)
  df = df[df.span_nbr !=-1 ][df.premise != ''] [df.hypotheis != '']
  df['bert_sent']= df.apply(lambda s: [s.premise,s.hypotheis], axis=1)
  return df

def create_completedf(data, refdf, max_neutral=50):
  cl = get_data(data, 0,'nda-1', 1)
  cols = list(cl.columns)
  df1= pd.DataFrame(columns=cols)
  for ind, row in refdf.iterrows():    
    df2 = get_data(data, row["idx"],row["nda_key"], max_neutral)
    df1=pd.concat([df1,df2])
  return df1



In [ ]:
#https://gist.github.com/jamescalam/7117aa92235a7f52141ad0654795aa48

from transformers import AutoTokenizer, AutoModel
import torch
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
def sent_embs(file, json, ke, premise=True):
  idx= flnmidx[file]
  ad =get_data(json, idx,ke, max_neutral=5000)
  if premise:
    sentences = ad.premise.values.tolist()
    '''
    spans= pdf_struct.predict(
                        format='paragraphs',
                        in_path=x[0],
                        model='PDFContractEnFeatureExtractor'
                      )
    sentences = pd.DataFrame(spans)[0].apply(preprocess).tolist()
    '''
  else:
    sentences = [ad[ad.hypotheis_key==ke].hypotheis.values[0]]

  # Load model from HuggingFace Hub
  tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/nli-bert-base')
  model = AutoModel.from_pretrained('sentence-transformers/nli-bert-base')

  # Tokenize sentences
  encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

  # Compute token embeddings
  with torch.no_grad():
      model_output = model(**encoded_input)

  # Perform pooling. In this case, max pooling.
  embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
  return sentences, embeddings

In [ ]:
kes=['nda-11', 'nda-16', 'nda-15', 'nda-10', 'nda-2', 'nda-1', 'nda-19', 'nda-12', 'nda-20', 'nda-3', 'nda-18', 'nda-7', 'nda-17', 'nda-8', 'nda-13', 'nda-5', 'nda-4']
idx= 25
ke= 'nda-4'
ad =get_data(data, idx,ke, max_neutral=3)
ad.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,docid,file_name,hypotheis_key,hypotheis,span_nbr,premise,choice,label,entl,cont,neut,bert_sent
40,114,GSAOP0309-Non-disclosure-agreement.pdf,nda-4,receiving party shall not use any confidential...,40,under this agreement the recipient undertakes ...,Entailment,0,1,0,0,[under this agreement the recipient undertakes...
52,114,GSAOP0309-Non-disclosure-agreement.pdf,nda-4,receiving party shall not use any confidential...,52,the confidential information is supplied to th...,Entailment,0,1,0,0,[the confidential information is supplied to t...
53,114,GSAOP0309-Non-disclosure-agreement.pdf,nda-4,receiving party shall not use any confidential...,53,the confidential information cannot be used to...,Entailment,0,1,0,0,[the confidential information cannot be used t...
157,114,GSAOP0309-Non-disclosure-agreement.pdf,nda-4,receiving party shall not use any confidential...,157,this agreement and the rights and obligations ...,Entailment,0,1,0,0,[this agreement and the rights and obligations...
148,114,GSAOP0309-Non-disclosure-agreement.pdf,nda-4,receiving party shall not use any confidential...,148,this agreement shall be governed and construed...,Neutral,2,0,0,1,[this agreement shall be governed and construe...


'Receiving Party shall not use any Confidential Information for any purpose other than the purposes stated in Agreement.'

In [ ]:
x= data['documents'][idx]['file_name']
x

'GSAOP0309-Non-disclosure-agreement.pdf'

In [ ]:
import faiss
import pdf_struct
index = faiss.IndexFlatL2(d)

sentences_contract , contract_embeddings = sent_embs(x,data,ke)
index.add(contract_embeddings.numpy())
sentences_hypt, xq = sent_embs(x,data,ke, premise=False)
print(len(sentences_contract))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


173


In [ ]:
k = 15
D, I = index.search(xq.numpy(), k)  # search
print(I) 
hits=[f'{sentences_contract[i]}' for i in I[0]]
#hits = [f'{i}: {sentences_contract[i]}' for i in I[0]]

hits


[[127 149   5 108 161 115  92 158  48  70   3 165   2 151 145]]


['no failure or delay by the gsa in exercising any of its rights under this agreement shall operate as a waiver thereof nor shall any single or partial exercise preclude any other or further exercise of such rights',
 'no provision of this agreement shall be construed to be an obligation by either party to disclose information to the other party or to enter into further agreements with the other',
 'in any case the recipient and authorised third parties shall not use the confidential information',
 'the recipient shall not itself nor authorise authorised third party or any third party to write publish or disseminate any description of the confidential information or elements of it such as its structure or content for so long as it is bound by this agreement',
 'paragraphs and above shall not apply to the extent that the recipient is required to retain any such confidential information by any applicable law rule or regulation or by any competent judicial or governmental body',
 'nothing

In [ ]:
print(lbl[ke])
for pm in ad[ad.choice !='Neutral']["premise"].values:
  print("I am looking for : ",pm)
  i=0
  for sc in hits:
    if pm==sc:      
      print (i, I[0][i], sc)
    i=i+1


Receiving Party shall not use any Confidential Information for any purpose other than the purposes stated in Agreement.
I am looking for :  under this agreement the recipient undertakes to use the confidential information solely for pursuing the purpose in accordance with the terms of this agreement
I am looking for :  the confidential information is supplied to the recipient solely and exclusively for the purpose
I am looking for :  the confidential information cannot be used totally or partially directly or indirectly for any other purpose than that defined in article above unless the gsa gives its prior written authorisation
12 2 the confidential information cannot be used totally or partially directly or indirectly for any other purpose than that defined in article above unless the gsa gives its prior written authorisation
I am looking for :  this agreement and the rights and obligations hereunder may not be transferred or assigned by the recipient without the prior written approva